In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Working with Obsesity Challenge data: Feeding embeddings into SVM model

In this notebook, we replicated what the paper discussed in 2.2 (partially) and 2.3:
1. Processing Obesity Challenge data
2. Models. This includes
   *  Construct a sparse baseline SVM model using bag-of-words as features
   *  Construct a second baseline SVM model using SVD vectors as features
   *  Construct the proposed SVM model, with vectors from the hidden layer of the previously trained NN model as features
   * Ablation study I: using vectors from the averaged layer of the NN model as features
   * Ablation study II: using Word2Vec embeddings directly as features
   * Ablation study III: using Doc2Vec embeddings directly as features
3. Evaluation of model performances
   * Standard Evaluation
   * Filtering out diseases with few positive labels, then evaluate 


In [ ]:
import xml.etree.ElementTree as ET
import os
#https://docs.python.org/3/library/xml.etree.elementtree.html
import pandas as pd
import re
import pickle
import numpy as np
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.decomposition import TruncatedSVD

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

In [ ]:
train_annotation_path='/content/drive/My Drive/Obesity challenge/obesity_standoff_annotations_training.xml'
train_annotation_path2='/content/drive/My Drive/Obesity challenge/obesity_standoff_annotations_training_addendum3.xml'
cui_path='/content/drive/My Drive/Obesity challenge/train_data_cuis/'

#load cui2token dict from MIMIC data
with open("drive/MyDrive/mimic-iii/token2int_dict.pkl", "rb") as fp:
  mimic_cui2token = pickle.load(fp)

with open("drive/MyDrive/mimic-iii/label2idx_dict.pkl", "rb") as fp:   #Pickling
  label2idx_dict = pickle.load(fp)

with open("drive/MyDrive/mimic-iii/cui_inputs.pkl", "rb") as fp:   #Pickling
  cui_inputs = pickle.load(fp)

# 1. Obesity Challenge Data Processor

In [ ]:
class obesityDataLoader:
  def __init__(self, annotation_path, annotation_path2, cui_path):
    # obesity data has 2 train files, so we need to read both of them
    self.train_annotation=[ET.parse(annotation_path), ET.parse(annotation_path2)] # xml tree containing labels for all patients
    self.cui_path=cui_path # dir of txts containing cuis for each patient
    self.diseases=self.get_diseases()
    self.diseases2int={self.diseases[i]:i for i in range(len(self.diseases))}
    self.label2int= {'Y':0, 'N':1, 'Q':2, 'U':3}
    self.doc2labels=[]
    self.token2int={}

  def get_diseases(self):
    diseases = []
    for train_annotation in self.train_annotation:
      for disease in train_annotation.iter('disease'):
        name = disease.attrib['name']
        diseases.append(name)
    diseases = list(set(diseases))
    diseases.sort(reverse = False ) 
    return diseases

  def map_annotation_to_labels(self):
    patient_results = {}
    for annotation_root in self.train_annotation:
      root = annotation_root.getroot()
      root_intuitive = root[0] # only intuitive tasks are needed in the paper
      for disease in root_intuitive.iter('disease'):
        dis_name = disease.attrib["name"]
        for patient in disease.iter('doc'):
          id = int(patient.attrib['id']) # use int id
          if id not in patient_results:
            patient_results[id] = [0] * len(self.diseases)
          res = patient.attrib['judgment']
          int_label = self.label2int[res]
          disease_id = self.diseases2int[dis_name]
          patient_results[id][disease_id] = int_label
    return patient_results   # this contains # patients x # possible diseases, dictionary: id to list of diseases   

  def set_cuis(self, mimic_cui_dict):   
      self.token2int= mimic_cui_dict
      
  def create_cui_input_sequence(self, tokens):
    input = []
    tokens_set = set(tokens)
    for token in tokens_set:
      if token in self.token2int:
        input.append(self.token2int[token])
      else:
        input.append(self.token2int['oov_word'])
    return input
  
  def map_annotation_to_labels_test(self, annotation_root):
      patient_results = {}
      root = annotation_root.getroot()
      root_intuitive = root[0] # only intuitive tasks are needed in the paper
      for disease in root_intuitive.iter('disease'):
        dis_name = disease.attrib["name"]
        for patient in disease.iter('doc'):
          id = int(patient.attrib['id']) # use int id
          if id not in patient_results:
            patient_results[id] = [0] * len(self.diseases)
          res = patient.attrib['judgment']
          int_label = self.label2int[res]
          disease_id = self.diseases2int[dis_name]
          patient_results[id][disease_id] = int_label
      return patient_results   # this contains # patients x # possible diseases, dictionary: id to list of diseases 

  def get_test_data(self, test_annotation_path, test_cui_path, raw=False):
    test_annotation = ET.parse(test_annotation_path)
    codes = []
    labels = []
    test_patient_labels = self.map_annotation_to_labels_test(test_annotation)
    for file in os.listdir(test_cui_path):
      text = open(os.path.join(test_cui_path, file)).read()     
      subj_id = int(re.findall('\d+(?=.txt)', file)[0])

      if raw:
        cui_tokens = text
      else:
        cuis = [token for token in text.split()]
        cui_tokens = self.create_cui_input_sequence(cuis)
      
      if subj_id in test_patient_labels:
        codes.append(cui_tokens)
        labels.append(test_patient_labels[subj_id])  
          
    diseases_label_dict ={}
    for disease in self.diseases:
      diseases_label_dict[disease] = [patient[self.diseases2int[disease]] for patient in labels]
    return codes, diseases_label_dict

  def get_train_data(self, raw=False):
    codes = []
    labels = []
    all_patient_labels = self.map_annotation_to_labels()

    for file in os.listdir(self.cui_path):
      text = open(os.path.join(self.cui_path, file)).read()
      subj_id = int(re.findall('\d+(?=.txt)', file)[0])
      if raw:
        cui_tokens = text
      else:
        cuis = [token for token in text.split()]
        cui_tokens = self.create_cui_input_sequence(cuis)
      if subj_id in all_patient_labels:
        codes.append(cui_tokens)
        labels.append(all_patient_labels[subj_id])
    diseases_label_dict ={}
    for disease in self.diseases:
      diseases_label_dict[disease] = [patient[self.diseases2int[disease]] for patient in labels]
    return codes, diseases_label_dict
    # diseases_label_dict: {"Asthma":[1,0,1,0,....], "CAD":[0,0,1,1,....], ...}

In [ ]:
train_data_loader = obesityDataLoader(train_annotation_path, train_annotation_path2, cui_path)
train_data_loader.set_cuis(mimic_cui2token)

In [ ]:
test_cui_path="/content/drive/My Drive/Obesity challenge/test_data_cuis/"
test_annotation_path='/content/drive/My Drive/Obesity challenge/obesity_standoff_annotations_test.xml'

# 2. Models

## 2.1 Sparse baseline Model

In [ ]:
#tfidf features
def count_and_tfidf(count_vectorizer, tfidf, data, test=False):
  if not test:
    data_count_matrix = count_vectorizer.fit_transform(data)
    data_tfidf = tfidf.fit_transform(data_count_matrix)
  else:
    data_count_matrix = count_vectorizer.transform(data)
    data_tfidf = tfidf.transform(data_count_matrix)
  return data_tfidf

#evaluation
def eval_p_r_and_f1(pred, y_test):
  p = precision_score(y_test, pred, average='macro')
  r = recall_score(y_test, pred, average='macro')
  f1 = f1_score(y_test, pred, average='macro')
  return p, r, f1

#run train and predict
def run_sparse_baseline_model(loader, disease, test_annotation_path, test_cui_path):
  x_train, y_train_dict = loader.get_train_data(raw=True) 
  y_train = y_train_dict[disease]
  x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=True) #TODO
  y_test = y_test_dict[disease]

  count_vectorizer = CountVectorizer(
    ngram_range=(1,1),
    stop_words='english',
    min_df=0,
    vocabulary=None,
    binary=False)

  tfidf = TfidfTransformer()

  train_tfidf = count_and_tfidf(count_vectorizer, tfidf, x_train)
  test_tfidf = count_and_tfidf(count_vectorizer, tfidf, x_test, test=True)

  clf = LinearSVC(class_weight='balanced')
  clf.fit(train_tfidf, y_train)
  pred = clf.predict(test_tfidf)

  p, r, f1 = eval_p_r_and_f1(pred, y_test)
  return p, r, f1

In [ ]:
p, r, f1 = run_sparse_baseline_model(train_data_loader, "Asthma", test_annotation_path, test_cui_path)
print(p, r, f1)

0.7553191489361701 0.6993355481727574 0.7185185185185186


## 2.2 SVD Baseline Model

In [ ]:
#SVD has same pipeline as sparse feature SVM
def run_svd_baseline_model(loader, disease, test_annotation_path, test_cui_path):
    x_train, y_train_dict = loader.get_train_data(raw=True) 
    y_train = y_train_dict[disease]
    x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=True) #TODO
    y_test = y_test_dict[disease]

    count_vectorizer = CountVectorizer(
    ngram_range=(1,1),
    stop_words='english',
    min_df=0,
    vocabulary=None,
    binary=False)

    tfidf = TfidfTransformer()

    train_tfidf = count_and_tfidf(count_vectorizer, tfidf, x_train)
    test_tfidf = count_and_tfidf(count_vectorizer, tfidf, x_test, test=True)

    svd = TruncatedSVD(n_components=300)
    svd_train_tfidf = svd.fit_transform(train_tfidf)
    svd_test_tfidf = svd.transform(test_tfidf)

    svd_clf = LinearSVC(class_weight='balanced')
    svd_clf.fit(svd_train_tfidf, y_train)
    svd_pred = svd_clf.predict(svd_test_tfidf)

    p, r, f1 = eval_p_r_and_f1(svd_pred, y_test)
    return p, r, f1

In [ ]:
p, r, f1 = run_svd_baseline_model(train_data_loader, "Asthma", test_annotation_path, test_cui_path)
print(p, r, f1)

0.7226704316256556 0.703765227021041 0.7119410887038552


## 2.3 Dense Neural Network Model

In [ ]:
# model configs
criterion = nn.BCELoss()
n_epochs = 75
batch_size = 50

maxlen = max([len(patient) for patient in cui_inputs])
emb_dim = len(mimic_cui2token)
n_class = len(label2idx_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class customDataset(Dataset):
    def __init__(self, x, y):
        self.x = x # shape n_sample x padded length
        self.y = y # shape n_sample x n classes [[0,1,0,0], [1,1,1,0]]
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        return(self.x[index], self.y[index])

def collate_fn(data):
    sequences, labels = zip(*data)

    n = len(sequences)
    y = torch.zeros((n, 100), dtype=torch.float).to(device)

    x = torch.zeros((n, maxlen), dtype=torch.long).to(device)
    
    for patient, cuis in enumerate(sequences):
      len_cuis = len(cuis)
      if len_cuis > maxlen:
        cuis = cuis[:maxlen]
        x[patient][:maxlen] = torch.tensor(cuis).to(device)
      else:
        x[patient][:len_cuis] = torch.tensor(cuis).to(device)
    return x, y
    
def get_hidden_states(loader, model):
    model.eval()
    hiddens_lst = []
    for current_x, current_y in loader:
      hiddens = model.get_hidden(current_x).cpu().detach().numpy()
      hiddens_lst.append(hiddens)
    hiddens_lst = np.vstack(hiddens_lst)
    return hiddens_lst #output should be 2d list of hidden states

def get_averaged_states(loader, model):
    model.eval()
    avg_lst = []
    for current_x, current_y in loader:
      hiddens = model.get_averaged(current_x).cpu().detach().numpy()
      avg_lst.append(hiddens)
    avg_lst = np.vstack(avg_lst)
    return avg_lst 


In [ ]:
class NN_representation(nn.Module):
  def __init__(self, in_dim, n_diseases, w2v_model=None):
    super(NN_representation, self).__init__()
    if w2v_model:
      weights = torch.FloatTensor(w2v_model.wv.vectors)
      self.emb = nn.Embedding(num_embeddings= in_dim, embedding_dim= 300).from_pretrained(weights)
    else:
      self.emb = nn.Embedding(num_embeddings= in_dim, embedding_dim= 300)
    self.avg = nn.AdaptiveMaxPool1d(1)
    self.hidden = nn.Linear(300, 1000)
    self.act1 = nn.ReLU()
    self.final = nn.Linear(1000, n_diseases)
    self.act2 = nn.Sigmoid()

  def get_hidden(self, x):
    temp = self.emb(x)
    temp = torch.permute(temp, (0,2,1))
    temp = self.avg(temp)
    temp = temp.squeeze(-1)
    h = self.hidden(temp)
    return h

  def get_averaged(self, x):
    temp = self.emb(x)
    temp = torch.permute(temp, (0,2,1))
    temp = self.avg(temp)
    temp = temp.squeeze(-1)
    return temp

  def forward(self, x):
    temp = self.emb(x)
    #print(f"after emb, {temp.shape}")
    temp = torch.permute(temp, (0,2,1))
    #print(f"after permute, {temp.shape}")
    temp = self.avg(temp)
    #print(f"after avg, {temp.shape}")
    temp = temp.squeeze(-1)
    #print(f"after squeeze, {temp.shape}")
    temp = self.hidden(temp)
    #print(f"after hidden, {temp.shape}")
    temp = self.act1(temp)
    #print(f"after relu, {temp.shape}")
    temp = self.final(temp)
    #print(f"after linear hidden, {temp.shape}")
    res = self.act2(temp)
    return res

In [ ]:
#run nn model and feed into svm
def run_dense_model(loader, disease, test_annotation_path, test_cui_path, model):
    x_train, y_train_dict = loader.get_train_data(raw=False) 
    y_train = y_train_dict[disease]
    x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=False) #TODO
    y_test = y_test_dict[disease]

    # model configs
    batch_size = 50

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_dataset = customDataset(x_train, y_train)
    test_dataset = customDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    train_hiddens = get_hidden_states(train_loader, model)
    test_hiddens = get_hidden_states(test_loader, model)

    dense_clf = LinearSVC(class_weight='balanced')
    dense_clf.fit(train_hiddens, y_train)
    dense_pred = dense_clf.predict(test_hiddens)

    p, r, f1 = eval_p_r_and_f1(dense_pred, y_test)
    return p, r, f1

In [ ]:
model =NN_representation(emb_dim, n_class).to(device)
# load model pretrained in another NNModels notebook
model.load_state_dict(torch.load("drive/MyDrive/mimic-iii/nnmodel_tuned_state_dict.pt"))
p, r, f1 = run_dense_model(train_data_loader, "Asthma", test_annotation_path, test_cui_path, model)
print(p, r, f1)

0.7718075539568345 0.7231450719822813 0.7413271480435659


## 2.4 Ablation Study I: Neural Network Model with vectors from averaged layer as embeddings

In [ ]:
def run_dense_model_avg_states(loader, disease, test_annotation_path, test_cui_path, model):
    x_train, y_train_dict = loader.get_train_data(raw=False) 
    y_train = y_train_dict[disease]
    x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=False) #TODO
    y_test = y_test_dict[disease]

    # model configs
    batch_size = 50

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_dataset = customDataset(x_train, y_train)
    test_dataset = customDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    train_hiddens = get_averaged_states(train_loader, model)
    test_hiddens = get_averaged_states(test_loader, model)

    dense_clf = LinearSVC(class_weight='balanced')
    dense_clf.fit(train_hiddens, y_train)
    dense_pred = dense_clf.predict(test_hiddens)

    p, r, f1 = eval_p_r_and_f1(dense_pred, y_test)
    return p, r, f1

In [ ]:
model =NN_representation(emb_dim, n_class).to(device)
model.load_state_dict(torch.load("drive/MyDrive/mimic-iii/nnmodel_state_dict.pt"))
p, r, f1 = run_dense_model_avg_states(train_data_loader, "Asthma", test_annotation_path, test_cui_path, model)
print(p, r, f1)

0.886988011988012 0.7859911406423035 0.8203781512605042


## 2.5 Ablation Study II: Word2Vec Embeddings into SVM Model

In [ ]:
def get_embeddings_vector(data, model):
  res=[]
  for doc in data:
    temp = [model.wv[str(token)] for token in doc]
    res.append(sum(temp)/len(temp))  
  return np.asarray(res)

In [ ]:
# load pretrained word2vec model (trained in the other notebook)
W2Vmodel = Word2Vec.load("drive/MyDrive/mimic-iii/word2vec.model")

In [ ]:
def run_w2v_embeddings_model(loader, disease, test_annotation_path, test_cui_path, model):
    x_train, y_train_dict = loader.get_train_data(raw=False) 
    y_train = y_train_dict[disease]
    x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=False) #TODO
    y_test = y_test_dict[disease]

    train_embeddings = get_embeddings_vector(x_train, model)
    test_embeddings = get_embeddings_vector(x_test, model)

    w2v_clf = LinearSVC(class_weight='balanced')
    w2v_clf.fit(train_embeddings, y_train)
    w2v_pred = w2v_clf.predict(test_embeddings)

    p, r, f1 = eval_p_r_and_f1(w2v_pred, y_test)
    return p, r, f1

## 2.6 Ablation Study III: Doc2Vec Embeddings into SVM Model

In [ ]:
def get_d2v_embeddings_vector(data, model):
  res=[]
  for doc in data:
    temp = [str(i) for i in doc]
    res.append(model.infer_vector(temp))  
  return np.asarray(res)

In [ ]:
# load pretrained doc2vec model (trained in the other notebook)
D2Vmodel = Doc2Vec.load("drive/MyDrive/mimic-iii/doc2vec.model")

In [ ]:
def run_d2v_embeddings_model(loader, disease, test_annotation_path, test_cui_path, model):
    x_train, y_train_dict = loader.get_train_data(raw=False) 
    y_train = y_train_dict[disease]
    x_test, y_test_dict = loader.get_test_data(test_annotation_path, test_cui_path, raw=False) #TODO
    y_test = y_test_dict[disease]

    train_embeddings = get_d2v_embeddings_vector(x_train, model)
    test_embeddings = get_d2v_embeddings_vector(x_test, model)

    w2v_clf = LinearSVC(class_weight='balanced')
    w2v_clf.fit(train_embeddings, y_train)
    w2v_pred = w2v_clf.predict(test_embeddings)

    p, r, f1 = eval_p_r_and_f1(w2v_pred, y_test)
    return p, r, f1

In [ ]:
p, r, f1 = run_d2v_embeddings_model(train_data_loader, "Asthma", test_annotation_path, test_cui_path, D2Vmodel)
print(p, r, f1)

0.6730769230769231 0.6644518272425248 0.6683695189442316


# Performance Evaluation:

## 3.1. Standard Evaluation

In [ ]:
#run evaluation on all diseases for all models

sparse_baseline_p = []
sparse_baseline_r = []
sparse_baseline_f = []

sparse_svd_p = []
sparse_svd_r = []
sparse_svd_f = []

dense_p = []
dense_r = []
dense_f = []

dense_avg_p = []
dense_avg_r = []
dense_avg_f = []

w2v_p = []
w2v_r = []
w2v_f = []

d2v_p = []
d2v_r = []
d2v_f = []

model =NN_representation(emb_dim, n_class).to(device)
model.load_state_dict(torch.load("drive/MyDrive/mimic-iii/nnmodel_state_dict.pt"))

for disease in train_data_loader.get_diseases():
  p, r, f1 = run_sparse_baseline_model(train_data_loader, disease, test_annotation_path, test_cui_path)
  sparse_baseline_p.append(p)
  sparse_baseline_r.append(r)
  sparse_baseline_f.append(f1)

  p, r, f1 = run_svd_baseline_model(train_data_loader, disease, test_annotation_path, test_cui_path)
  sparse_svd_p.append(p)
  sparse_svd_r.append(r)
  sparse_svd_f.append(f1)

  p, r, f1 = run_dense_model(train_data_loader, disease, test_annotation_path, test_cui_path, model)
  dense_p.append(p)
  dense_r.append(r)
  dense_f.append(f1)

  p, r, f1 = run_dense_model_avg_states(train_data_loader, disease, test_annotation_path, test_cui_path, model)
  dense_avg_p.append(p)
  dense_avg_r.append(r)
  dense_avg_f.append(f1)

  p, r, f1 = run_w2v_embeddings_model(train_data_loader, disease, test_annotation_path, test_cui_path, W2Vmodel)
  w2v_p.append(p)
  w2v_r.append(r)
  w2v_f.append(f1)

  p, r, f1 = run_d2v_embeddings_model(train_data_loader, disease, test_annotation_path, test_cui_path, D2Vmodel)
  d2v_p.append(p)
  d2v_r.append(r)
  d2v_f.append(f1)  

In [ ]:
sparse_baseline_p[:5], sparse_baseline_r[:5], sparse_baseline_f[:5]

In [ ]:
sparse_svd_p[:5], sparse_svd_r[:5], sparse_svd_f[:5]

In [ ]:
dense_p[:5], dense_r[:5], dense_f[:5]

In [ ]:
dense_avg_p[:5], dense_avg_r[:5], dense_avg_f[:5]

In [ ]:
w2v_p[:5], w2v_r[:5], w2v_f[:5]

In [ ]:
d2v_p[:5], d2v_r[:5], d2v_f[:5]

In [ ]:
np.mean(sparse_baseline_f), np.mean(sparse_svd_f), np.mean(dense_f), np.mean(dense_avg_f), np.mean(w2v_f), np.mean(d2v_f)

In [ ]:
np.mean(sparse_baseline_p), np.mean(sparse_svd_p), np.mean(dense_p), np.mean(dense_avg_p), np.mean(w2v_p), np.mean(d2v_p)

In [ ]:
np.mean(sparse_baseline_r), np.mean(sparse_svd_r), np.mean(dense_r), np.mean(dense_avg_r), np.mean(w2v_r), np.mean(d2v_r)

In [ ]:
#saving results

# np.save("drive/MyDrive/mimic-iii/sparse_baseline_f.npy", sparse_baseline_f)
# np.save("drive/MyDrive/mimic-iii/sparse_svd_f.npy", sparse_svd_f)
# np.save("drive/MyDrive/mimic-iii/dense_f.npy", dense_f)

# np.save("drive/MyDrive/mimic-iii/sparse_baseline_p.npy", sparse_baseline_p)
# np.save("drive/MyDrive/mimic-iii/sparse_svd_p.npy", sparse_svd_p)
# np.save("drive/MyDrive/mimic-iii/dense_p.npy", dense_p)

# np.save("drive/MyDrive/mimic-iii/sparse_baseline_r.npy", sparse_baseline_r)
# np.save("drive/MyDrive/mimic-iii/sparse_svd_r.npy", sparse_svd_r)
# np.save("drive/MyDrive/mimic-iii/dense_r.npy", dense_r)